![Course header](../assets/img/header.png)

# A2 — Parallel Computing with Dask
Optional reference — scale your EO workflows beyond laptop RAM

This appendix introduces **Dask**, a library that adds chunked, parallel execution to NumPy and xarray.
You will create a local cluster, compute NDVI in parallel, and learn the lazy-evaluation pattern.

## Learning Objectives

By the end of this notebook you will be able to:

- Explain why Dask is needed for large-scale EO analysis
- Create a local Dask cluster and open the Dashboard
- Open raster data with `chunks=` for lazy, parallel processing
- Understand the difference between lazy graphs and `.compute()`
- Follow best practices for chunk sizing

Tooling:
- dask.distributed
- xarray + rioxarray
- matplotlib

⏱️ Estimated time: **30 – 45 minutes**

---

## Table of contents

1. Why Dask?
2. Setup — create a local cluster
3. Lazy evaluation with xarray + Dask
4. Parallel NDVI example
5. Best practices
6. Scaling beyond your laptop
7. Cleanup
8. Exercises
9. Recap

---

## 1) Why Dask?

A single Sentinel-2 tile at 10 m resolution is ~1 GB.  
A year of data over a country can easily exceed **hundreds of GB**.

| Challenge | Dask’s solution |
|-----------|------------------|
| Data doesn’t fit in RAM | **Chunking** — process piece by piece |
| Loading data you don’t need | **Lazy evaluation** — load on demand |
| Single-core bottleneck | **Parallelisation** — use all cores |

Dask integrates seamlessly with xarray:  
open data with `chunks=` and every subsequent xarray operation runs in parallel automatically.

---

## 2) Setup — create a local cluster

In [ ]:
import numpy as np
import xarray as xr
import rioxarray
import matplotlib.pyplot as plt

In [ ]:
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(
    n_workers=4,
    threads_per_worker=1,
    memory_limit='2GB',
)
client = Client(cluster)
client  # click the Dashboard link to monitor workers

### The Dask Dashboard

Click the **Dashboard link** above. It shows:

- **Task Stream** — what each worker is doing over time
- **Progress** — overall bar for the current computation
- **Memory** — RAM usage per worker

> 💡 **Tip:** In JupyterLab the Dask sidebar (orange icon) lets you embed dashboard panels.

---

## 3) Lazy evaluation with xarray + Dask

When you open data with `chunks=`, xarray stores **Dask arrays** instead of NumPy arrays.  
Operations build a task graph but **don’t execute** until you call `.compute()`.

In [ ]:
# Open a COG with chunking enabled — nothing is loaded yet
cog_url = (
    'https://sentinel-cogs.s3.us-west-2.amazonaws.com/'
    'sentinel-s2-l2a-cogs/32/T/PS/2024/12/'
    'S2B_32TPS_20241228_0_L2A/B04.tif'
)

red = rioxarray.open_rasterio(cog_url, chunks='auto')

print(f'Shape:  {red.shape}')
print(f'Chunks: {red.chunks}')
print(f'Backed by: {type(red.data).__name__}')  # dask.array
red

In [ ]:
# Chain lazy operations — NO data is loaded
red_sq = red.squeeze('band', drop=True)
patch  = red_sq.isel(x=slice(2000, 4000), y=slice(2000, 4000))
refl   = patch / 10_000.0
mean_r = refl.mean()

print(f'Mean (lazy): {mean_r}')  # still a Dask scalar

### Triggering computation: `.compute()`

Call `.compute()` to execute the graph.  Watch the Dashboard!

In [ ]:
%%time
result = mean_r.compute()
print(f'Mean reflectance: {result:.4f}')

---

## 4) Parallel NDVI example

Load Red and NIR bands with chunking, compute NDVI, and plot — all in parallel.

In [ ]:
base_url = (
    'https://sentinel-cogs.s3.us-west-2.amazonaws.com/'
    'sentinel-s2-l2a-cogs/32/T/PS/2024/12/'
    'S2B_32TPS_20241228_0_L2A'
)

red = rioxarray.open_rasterio(
    f'{base_url}/B04.tif', chunks={'x': 1024, 'y': 1024}
).squeeze('band', drop=True)

nir = rioxarray.open_rasterio(
    f'{base_url}/B08.tif', chunks={'x': 1024, 'y': 1024}
).squeeze('band', drop=True)

print(f'Red chunks: {red.chunks}')
print(f'NIR chunks: {nir.chunks}')

In [ ]:
# NDVI — still lazy
ndvi = (nir - red) / (nir + red)
ndvi.name = 'ndvi'

print(f'Backed by: {type(ndvi.data).__name__}')
print(f'Chunks:    {len(ndvi.chunks[0])} × {len(ndvi.chunks[1])}')
ndvi

In [ ]:
%%time
# Coarsen for faster demo, then compute
ndvi_small = ndvi.coarsen(x=4, y=4, boundary='trim').mean()
ndvi_computed = ndvi_small.compute()

fig, ax = plt.subplots(figsize=(8, 8))
ndvi_computed.plot(ax=ax, cmap='RdYlGn', vmin=-0.5, vmax=0.8)
ax.set_title('NDVI (computed with Dask)')
ax.set_aspect('equal')
plt.tight_layout()
plt.show()

---

## 5) Best practices

### When to call `.compute()`

| Situation | Approach |
|-----------|----------|
| Building a processing chain | Stay **lazy** — chain operations |
| Checking an intermediate value | `.compute()` on a small subset |
| Final result | `.compute()` |
| Saving to file | `.to_netcdf()` / `.to_zarr()` trigger compute |
| Plotting | `.plot()` triggers compute |

### Chunk-size guidelines

- Target **100 MB – 1 GB** per chunk (rule of thumb)
- Too small → scheduling overhead
- Too large → memory pressure
- Align chunks with COG tile size or Zarr chunk layout

In [ ]:
# Check actual chunk size in MB
chunk_mb = (
    red.data.chunksize[0] * red.data.chunksize[1]
    * red.dtype.itemsize / 1e6
)
print(f'Chunk size: {chunk_mb:.1f} MB')

### Common mistakes

| Mistake | Why it’s bad |
|---------|-------------|
| Calling `.values` on a huge array | Loads everything into memory |
| Rechunking repeatedly | Expensive shuffle |
| Millions of tiny tasks | Scheduler overhead |
| Forgetting to close the client | Resource leak |

---

## 6) Scaling beyond your laptop

The same code works on bigger systems — only the cluster definition changes:

| Environment | Cluster type | Package |
|-------------|--------------|----------|
| Laptop | `LocalCluster` | dask.distributed |
| HPC (SLURM / PBS) | `SLURMCluster` | dask-jobqueue |
| Kubernetes | `KubeCluster` | dask-kubernetes |
| Managed cloud | `Gateway` | Dask Gateway / Coiled |

```python
# Example: SLURM cluster on an HPC
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(cores=4, memory='16GB', walltime='01:00:00')
cluster.scale(jobs=10)
client = Client(cluster)
```

---

## 7) Cleanup

In [ ]:
client.close()
cluster.close()
print('✅ Dask cluster closed')

---

## 8) Exercises

### ✅ Try it — Chunk comparison

1. Re-open the Red band COG with `chunks={'x': 512, 'y': 512}`.
2. Print the number of chunks (`len(red.chunks[0]) * len(red.chunks[1])`).
3. Compare with `chunks={'x': 2048, 'y': 2048}` — how many chunks now?
4. Which would you expect to be faster? Why?

<details><summary>Show solution</summary>

```python
red_512 = rioxarray.open_rasterio(
    f"{base_url}/B04.tif", chunks={"x": 512, "y": 512}
).squeeze("band", drop=True)
n_512 = len(red_512.chunks[0]) * len(red_512.chunks[1])

red_2048 = rioxarray.open_rasterio(
    f"{base_url}/B04.tif", chunks={"x": 2048, "y": 2048}
).squeeze("band", drop=True)
n_2048 = len(red_2048.chunks[0]) * len(red_2048.chunks[1])

print(f"512×512  → {n_512:,} chunks")
print(f"2048×2048 → {n_2048:,} chunks")
print("Fewer, larger chunks usually run faster (less scheduling overhead).")
```

</details>

In [ ]:
# ✅ Open red band with 512 and 2048 chunk sizes — compare chunk counts


### ✅ Try it — Parallel statistics

Compute the **min, max, and standard deviation** of the NDVI array (before coarsening). Use `.compute()` once on all three.

<details><summary>Show solution</summary>

```python
ndvi_stats = xr.Dataset({
    "min": ndvi.min(),
    "max": ndvi.max(),
    "std": ndvi.std(),
}).compute()

print(ndvi_stats)
```

</details>

In [ ]:
# ✅ Compute min, max, and std of ndvi in one .compute() call


### 🧠 Checkpoint

**Q1.** What does `chunks='auto'` do when opening a raster?

- A) Loads the entire file into a NumPy array
- B) Lets Dask choose chunk sizes that match the file's internal tiling
- C) Disables chunking

**Q2.** When does a Dask computation actually run?

- A) As soon as you write the expression
- B) When you call `.compute()` (or plot / save)
- C) It never runs automatically

**Q3.** What is a good target chunk size?

- A) 1 KB
- B) 100 MB – 1 GB
- C) 50 GB

<details><summary>Show answers</summary>

1. **B** — `chunks='auto'` lets Dask pick chunk sizes aligned with the file's internal tiling (COG block size or Zarr chunks).
2. **B** — Dask is lazy; computation happens only when you call `.compute()`, `.values`, `.plot()`, or save to a file.
3. **B** — Target 100 MB – 1 GB per chunk. Too small = scheduling overhead; too large = memory pressure.

</details>

---

## 9) Recap

| Concept | Key takeaway |
|---------|-------------|
| Dask | Chunked parallel execution for NumPy / xarray |
| `LocalCluster` | Run workers on your own machine |
| `chunks=` | Turns xarray arrays into lazy Dask graphs |
| `.compute()` | Triggers actual execution |
| Dashboard | Monitor workers, tasks, and memory |

💡 **Stay lazy as long as possible** — chain operations before computing.

### Further reading

- [Dask documentation](https://docs.dask.org/)
- [xarray with Dask](https://docs.xarray.dev/en/stable/user-guide/dask.html)
- [Dask best practices](https://docs.dask.org/en/stable/best-practices.html)
- [Pangeo](https://pangeo.io/) — community using Dask for geoscience